# Batch Data Ingestion - Demo

**Cel szkoleniowy:** Opanowanie technik idempotentnego ładowania danych batch do Delta Lake z użyciem COPY INTO, CTAS i Lakeflow Connect.

**Zakres tematyczny:**
- COPY INTO (idempotent batch load)
- CTAS (CREATE TABLE AS SELECT)
- Lakeflow Connect (managed connectors)
- File formats: JSON, CSV, Parquet, Avro
- Schema inference vs enforcement
- Error handling (badRecordsPath, rescue columns)
- Incremental batch patterns

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Delta Lake & Lakehouse Architecture
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers
- **Zależności**: Wykonany notebook 01_delta_lake_operations.ipynb

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie różnych metod batch ingestion do Delta Lake i kiedy je stosować.

**Podstawowe pojęcia:**

- **COPY INTO**: Idempotentna komenda do ładowania plików z zewnętrznych lokalizacji (cloud storage). Automatycznie śledzi już przetworzone pliki, zapobiegając duplikacjom.

- **CTAS (CREATE TABLE AS SELECT)**: Tworzy nową tabelę na podstawie zapytania SELECT. Przydatne do jednorazowego ładowania lub transformacji danych.

- **Lakeflow Connect**: Managed connectors do popularnych źródeł danych (Salesforce, SQL Server, etc.) - upraszcza integracje bez konieczności pisania custom code.

- **Idempotency**: Właściwość operacji, która daje ten sam rezultat niezależnie od liczby wykonań. Kluczowa w pipeline'ach, które mogą być uruchamiane wielokrotnie.

**Dlaczego to ważne?**

Batch ingestion to fundament data lakehouse - większość danych trafia do platformy przez regularne, zaplanowane ładowania. Dobra strategia ingestion zapewnia:
- Brak duplikacji danych
- Możliwość reprocessingu bez skutków ubocznych
- Efektywne wykorzystanie zasobów
- Łatwe debugowanie i monitoring

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [ ]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
import os

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")


print(f"\n=== Ścieżki do danych ===")
print(f"Customers CSV: {CUSTOMERS_CSV}")
print(f"Orders JSON: {ORDERS_JSON}")
print(f"Products Parquet: {PRODUCTS_PARQUET}")

## Sekcja 1: COPY INTO - Idempotent Batch Load

**Wprowadzenie teoretyczne:**

COPY INTO to najbardziej rekomendowana metoda dla batch ingestion w Delta Lake. Kluczowe cechy:

**Kluczowe pojęcia:**

- **Idempotency**: COPY INTO automatycznie śledzi przetworzone pliki w transaction log. Ponowne uruchomienie nie powoduje duplikacji.

- **File tracking**: Delta Lake zapisuje checksums przetworzonych plików. Tylko nowe pliki są ładowane przy kolejnych uruchomieniach.

- **FORMAT options**: Wspiera CSV, JSON, PARQUET, AVRO, ORC, TEXT, BINARYFILE.

- **COPY_OPTIONS**: Parametry specyficzne dla formatu (delimiter, header, dateFormat, etc.).

**Zastosowanie praktyczne:**
- Incremental batch loads (nowe pliki codziennie/co godzinę)
- Reprocessing bez duplikacji
- Data lake ingestion z S3/ADLS/GCS

### Przykład 1.1: COPY INTO from CSV

**Cel:** Załadować dane customers z CSV do Delta table używając COPY INTO.

**Podejście:**
1. Utwórz target Delta table
2. Użyj COPY INTO z opcjami CSV
3. Sprawdź idempotency (uruchom ponownie)

In [ ]:
# Przykład 1.1 - COPY INTO from CSV

TABLE_NAME = f"{BRONZE_SCHEMA}.customers_batch"

# Krok 1: Utwórz target table (jeśli nie istnieje)
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
  customer_id STRING,
  first_name STRING,
  last_name STRING,
  email STRING,
  phone STRING,
  address STRING,
  city STRING,
  state STRING,
  zip_code STRING,
  customer_segment STRING,
  registration_date DATE,
  _ingestion_timestamp TIMESTAMP
) USING DELTA
""")

print(f"✓ Tabela {TABLE_NAME} gotowa")

# Krok 2: COPY INTO z opcjami CSV
result = spark.sql(f"""
COPY INTO {TABLE_NAME}
FROM '{CUSTOMERS_CSV}'
FILEFORMAT = CSV
FORMAT_OPTIONS (
  'header' = 'true',
  'inferSchema' = 'false',
  'delimiter' = ','
)
COPY_OPTIONS (
  'mergeSchema' = 'false'
)
""")

display(result)

# Krok 3: Sprawdź liczbę załadowanych rekordów
count = spark.table(TABLE_NAME).count()
print(f"\n✓ Załadowano {count} rekordów do {TABLE_NAME}")

**Wyjaśnienie:**

COPY INTO automatycznie:
- Śledzi przetworzone pliki w `_delta_log`
- Pomija pliki już załadowane (idempotency)
- Zapisuje metadane: `num_affected_rows`, `num_inserted_rows`

FORMAT_OPTIONS:
- `header='true'`: Pierwsza linia to nagłówki kolumn
- `inferSchema='false'`: Używamy zdefiniowanego schematu tabeli
- `delimiter=','`: Separator CSV

Uruchom tę komórkę ponownie - zobaczysz, że COPY INTO nie załaduje duplikatów!

### Przykład 1.2: COPY INTO z transformacją (SELECT)

**Cel:** Załadować dane z transformacją podczas COPY INTO - dodanie audit columns.

**Podejście:**
1. Użyj SELECT w COPY INTO do transformacji
2. Dodaj `_ingestion_timestamp` i `_source_file`

In [ ]:
# Przykład 1.2 - COPY INTO z SELECT transformation

TABLE_ORDERS = f"{BRONZE_SCHEMA}.orders_batch"

# Utwórz tabelę
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TABLE_ORDERS} (
  order_id STRING,
  customer_id STRING,
  order_date DATE,
  total_amount DECIMAL(10,2),
  payment_method STRING,
  _ingestion_timestamp TIMESTAMP,
  _source_file STRING
) USING DELTA
""")

# COPY INTO z SELECT
result = spark.sql(f"""
COPY INTO {TABLE_ORDERS}
FROM (
  SELECT 
    order_id,
    customer_id,
    TO_DATE(order_date) as order_date,
    CAST(total_amount AS DECIMAL(10,2)) as total_amount,
    payment_method,
    current_timestamp() as _ingestion_timestamp,
    _metadata.file_path as _source_file
  FROM '{ORDERS_JSON}'
)
FILEFORMAT = JSON
""")

display(result)




# Przykład 3.1 - Schema Inference vs Enforcement

print("=== 1. Schema Inference (automatyczne wykrywanie) ===\n")

# Metoda 1: Schema Inference
df_inferred = spark.read.csv(CUSTOMERS_CSV, header=True, inferSchema=True)

print("Schema z inference:")
df_inferred.printSchema()
print(f"Liczba rekordów: {df_inferred.count()}\n")

# Pokaż przykładowe dane
display(df_inferred.limit(3))

print("\n=== 2. Explicit Schema (zdefiniowany schema) ===\n")

# Metoda 2: Explicit Schema (ZALECANE dla production)
schema_explicit = StructType([
  StructField("customer_id", StringType(), False),
  StructField("first_name", StringType(), True),
  StructField("last_name", StringType(), True),
  StructField("email", StringType(), True),
  StructField("phone", StringType(), True),
  StructField("address", StringType(), True),
  StructField("city", StringType(), True),
  StructField("state", StringType(), True),
  StructField("zip_code", StringType(), True),
  StructField("customer_segment", StringType(), True),
  StructField("registration_date", DateType(), True)
])

df_explicit = spark.read.schema(schema_explicit).csv(CUSTOMERS_CSV, header=True)

print("Schema explicit:")
df_explicit.printSchema()
print(f"Liczba rekordów: {df_explicit.count()}\n")

display(df_explicit.limit(3))

# Porównanie
print("\n=== Porównanie ===")
print("Schema Inference:")
print("  ✅ Szybkie do prototyping")
print("  ❌ Może błędnie wykryć typy (np. INT zamiast STRING)")
print("  ❌ Skanuje dane (wolniejsze dla dużych plików)")
print("\nExplicit Schema:")
print("  ✅ Precyzyjne typy danych")
print("  ✅ Walidacja podczas read")
print("  ✅ Dokumentacja w kodzie")
print("  ✅ Szybsze (no scan needed)")


# Przykład 4.1 - Error Handling z badRecordsPath

# Krok 1: Setup bad records path
BAD_RECORDS_PATH = f"/tmp/{raw_user}/bad_records"

# Wyczyść folder jeśli istnieje (dla demo)
try:
    dbutils.fs.rm(BAD_RECORDS_PATH, True)
except:
    pass

print(f"Bad records path: {BAD_RECORDS_PATH}")

# Krok 2: Utwórz tabelę z _corrupt_record column
TABLE_ERRORS = f"{BRONZE_SCHEMA}.customers_with_validation"

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TABLE_ERRORS} (
  customer_id STRING,
  first_name STRING,
  last_name STRING,
  email STRING,
  phone STRING,
  address STRING,
  city STRING,
  state STRING,
  zip_code STRING,
  customer_segment STRING,
  registration_date STRING,
  _corrupt_record STRING,
  _ingestion_timestamp TIMESTAMP
) USING DELTA
""")

# Krok 3: Wczytaj dane z error handling
df_with_errors = (spark.read
  .format("csv")
  .option("header", "true")
  .option("mode", "PERMISSIVE")
  .option("columnNameOfCorruptRecord", "_corrupt_record")
  .option("badRecordsPath", BAD_RECORDS_PATH)
  .load(CUSTOMERS_CSV)
  .withColumn("_ingestion_timestamp", F.current_timestamp())
)

# Zapisz do tabeli
df_with_errors.write.mode("append").saveAsTable(TABLE_ERRORS)

print(f"✓ Dane załadowane do {TABLE_ERRORS}")

# Krok 4: Analiza błędnych rekordów
print("\n=== Statystyki ===")
total_records = spark.table(TABLE_ERRORS).count()
corrupt_records = spark.table(TABLE_ERRORS).filter(F.col("_corrupt_record").isNotNull()).count()
valid_records = total_records - corrupt_records

print(f"Łącznie rekordów: {total_records}")
print(f"Poprawnych: {valid_records}")
print(f"Błędnych: {corrupt_records}")

# Jeśli są błędne rekordy, pokaż je
if corrupt_records > 0:
    print("\n⚠️ Błędne rekordy:")
    display(spark.table(TABLE_ERRORS).filter(F.col("_corrupt_record").isNotNull()))
    
    # Sprawdź bad records folder
    try:
        bad_files = dbutils.fs.ls(BAD_RECORDS_PATH)
        print(f"\n📁 Bad records zapisane w: {BAD_RECORDS_PATH}")
        for file in bad_files:
            print(f"  - {file.name} ({file.size} bytes)")
    except:
        print("\n✓ Brak plików w bad records folder (wszystkie dane poprawne)")
else:
    print("\n✅ Wszystkie rekordy poprawne!")


# Przykład 5.1 - CTAS dla agregacji

AGG_TABLE = f"{SILVER_SCHEMA}.customer_segment_summary"

print(f"=== Tworzenie tabeli agregacyjnej: {AGG_TABLE} ===\n")

# Krok 1: CTAS z agregacją
spark.sql(f"""
CREATE OR REPLACE TABLE {AGG_TABLE}
USING DELTA
COMMENT 'Customer segmentation summary - created via CTAS'
AS
SELECT 
  customer_segment,
  COUNT(*) as customer_count,
  COUNT(DISTINCT state) as states_count,
  MIN(registration_date) as first_registration,
  MAX(registration_date) as last_registration,
  current_timestamp() as snapshot_timestamp
FROM {TABLE_NAME}
WHERE customer_segment IS NOT NULL
GROUP BY customer_segment
ORDER BY customer_count DESC
""")

print(f"✓ Tabela {AGG_TABLE} utworzona\n")

# Krok 2: Pokaż wyniki
print("=== Summary po segmentach ===")
display(spark.table(AGG_TABLE))

# Krok 3: Sprawdź metadane tabeli
print("\n=== Metadane tabeli ===")
spark.sql(f"DESCRIBE EXTENDED {AGG_TABLE}").show(truncate=False)

# Krok 4: Historia tabeli
print("\n=== Historia tabeli (CTAS creates version 0) ===")
display(spark.sql(f"DESCRIBE HISTORY {AGG_TABLE}").limit(5))


# Przykład 6.1 - Lakeflow Connect Setup Guide

print("""
╔═══════════════════════════════════════════════════════════════════════════╗
║                    LAKEFLOW CONNECT - SETUP GUIDE                         ║
╚═══════════════════════════════════════════════════════════════════════════╝

📍 KROK 1: Nawigacja w UI
───────────────────────────────────────────────────────────────────────────
1. Otwórz Databricks workspace
2. Z lewego menu wybierz: Workflows
3. Kliknij przycisk "Create" → "Lakeflow"

📍 KROK 2: Wybór Connectora
───────────────────────────────────────────────────────────────────────────
Dostępne opcje:
  • Salesforce (CRM data, custom objects)
  • SQL Server (transactional databases, CDC)
  • PostgreSQL/MySQL (open-source databases)
  • Google Analytics 4 (web analytics)
  • ServiceNow (ITSM data)
  • Custom REST API

📍 KROK 3: Konfiguracja Connection
───────────────────────────────────────────────────────────────────────────
Przykład dla SQL Server:

  Connection Name: sqlserver_prod_connection
  Host: sql-server.example.com:1433
  Database: Adventureworks
  Authentication: SQL Server Authentication
  Username: lakehouse_reader
  Password: *** (stored in secret scope)
  
  ✓ Test Connection → Success

📍 KROK 4: Wybór Tabel/Objects
───────────────────────────────────────────────────────────────────────────
Zaznacz tabele do sync:
  ✓ Sales.Orders
  ✓ Sales.Customers  
  ✓ Sales.OrderDetails
  ✓ Production.Products

📍 KROK 5: Destination Configuration
───────────────────────────────────────────────────────────────────────────
  Target Catalog: """ + CATALOG + """
  Target Schema: """ + BRONZE_SCHEMA + """
  Table Prefix: sqlserver_
  
  Result tables:
    • """ + CATALOG + "." + BRONZE_SCHEMA + """.sqlserver_orders
    • """ + CATALOG + "." + BRONZE_SCHEMA + """.sqlserver_customers
    • """ + CATALOG + "." + BRONZE_SCHEMA + """.sqlserver_orderdetails
    • """ + CATALOG + "." + BRONZE_SCHEMA + """.sqlserver_products

📍 KROK 6: Sync Configuration
───────────────────────────────────────────────────────────────────────────
  Sync Mode: Incremental (CDC if supported)
  Schedule: Every 1 hour
  CDC Method: Change Tracking (SQL Server built-in)
  Initial Load: Full snapshot
  
  Advanced Options:
    • mergeSchema: enabled (handle new columns)
    • badRecordsPath: /mnt/bad_records/sqlserver/
    • Parallelism: 4 workers

📍 KROK 7: Monitoring & Alerting
───────────────────────────────────────────────────────────────────────────
  Enable monitoring: ✓
  Alert on failure: team-data-engineering@company.com
  Alert on schema change: yes
  Log retention: 30 days

📍 KROK 8: Deploy
───────────────────────────────────────────────────────────────────────────
  Review configuration → Deploy

  Status: Running
  Next sync: 2024-01-15 14:00:00 UTC
  Last successful sync: 2024-01-15 13:00:00 UTC
  Records synced (last run): 1,234 inserts, 456 updates, 12 deletes

═══════════════════════════════════════════════════════════════════════════

💡 ZALETY LAKEFLOW CONNECT:
  ✅ No custom code - konfiguracja przez UI
  ✅ Automatic CDC - tylko zmienione dane
  ✅ Schema evolution - automatyczne nowe kolumny
  ✅ Built-in retry logic - error handling
  ✅ Monitoring dashboard - real-time status
  ✅ Unity Catalog integration - security & governance

💡 USE CASES:
  • Salesforce → Lakehouse (CRM analytics)
  • SQL Server → Delta Lake (transactional data)
  • Google Analytics → Analytics (web data)
  • REST APIs → Bronze layer (custom integrations)

💡 ALTERNATIVE: Custom Code
  Jeśli Lakeflow Connect nie wspiera Twojego źródła:
  • Użyj PySpark + JDBC connector
  • Implementuj własną CDC logic
  • Scheduled notebook jobs
  
  Ale: Więcej code = więcej maintenance!

═══════════════════════════════════════════════════════════════════════════
""")

# Symulacja: sprawdź czy tabele z Lakeflow Connect istnieją
print("\n📊 Sprawdzanie tabel z Lakeflow Connect (symulacja):\n")

simulated_tables = [
    f"{BRONZE_SCHEMA}.sqlserver_orders",
    f"{BRONZE_SCHEMA}.sqlserver_customers",
]

for table in simulated_tables:
    full_table = f"{CATALOG}.{table}"
    try:
        exists = spark.catalog.tableExists(full_table)
        if exists:
            count = spark.table(full_table).count()
            print(f"  ✅ {table}: {count} rekordów")
        else:
            print(f"  ⚠️  {table}: nie istnieje (wymaga konfiguracji Lakeflow)")
    except:
        print(f"  ⚠️  {table}: nie istnieje (wymaga konfiguracji Lakeflow)")

print("\n💡 Aby skonfigurować Lakeflow Connect, przejdź do UI: Workflows → Create → Lakeflow")


## Sekcja 8: Troubleshooting & Common Issues

**Wprowadzenie:**

Podczas batch ingestion napotkasz różne problemy. Oto najczęstsze i ich rozwiązania.

### Problem 1: "File already processed" - COPY INTO idempotency

**Objawy:**
```
AnalysisException: File /path/to/file.parquet has already been processed
```
- COPY INTO odmawia przetworzenia pliku
- Chcesz wymusić reprocessing (np. po poprawieniu danych)

**Przyczyna:**
Delta Lake trackuje przetworzone pliki w transaction log. Checksum pliku jest zapisany.

**Rozwiązanie 1: Clear tracking (produkcja ostrożnie!)**
```sql
-- Wyczyść tracking dla konkretnej tabeli
DELETE FROM table WHERE _source_file LIKE '%filename.parquet%';

-- Następnie COPY INTO zadziała ponownie
```

**Rozwiązanie 2: Użyj nowego pliku**
```bash
# Zmień plik (inny checksum)
cp old_file.parquet new_file_v2.parquet
# COPY INTO wykryje nowy plik
```

**Rozwiązanie 3: VACUUM + retry (ostateczność)**
```sql
-- UWAGA: To usuwa history! Użyj tylko dla testów
ALTER TABLE table_name 
SET TBLPROPERTIES ('delta.deletedFileRetentionDuration' = 'interval 0 hours');

VACUUM table_name RETAIN 0 HOURS;

-- Teraz COPY INTO może przetwarzać ponownie
```

### Problem 2: Schema Mismatch Errors

**Objawy:**
```
AnalysisException: Cannot cast 'customer_id' from STRING to INT
Schema mismatch: expected INT, found STRING
```

**Przyczyna:**
- Explicit schema w tabeli nie pasuje do source data
- Schema inference wykrył inny typ niż expected

**Rozwiązanie 1: CAST w SELECT**
```sql
COPY INTO table
FROM (
  SELECT 
    CAST(customer_id AS STRING) as customer_id,  -- Force cast
    CAST(amount AS DECIMAL(10,2)) as amount
  FROM 'path'
)
```

**Rozwiązanie 2: mergeSchema dla nowych kolumn**
```sql
COPY INTO table
FROM 'path'
COPY_OPTIONS ('mergeSchema' = 'true')  -- Auto-add new columns
```

**Rozwiązanie 3: Recreate table z poprawnym schema**
```sql
DROP TABLE IF EXISTS table_name;

CREATE TABLE table_name (
  customer_id STRING,  -- Correct type
  amount DECIMAL(10,2)
) USING DELTA;
```

### Problem 3: Bad Records nie są zapisywane

**Objawy:**
- badRecordsPath folder jest pusty
- Corrupt records znikają bez trace

**Przyczyna:**
- Mode ustawiony na DROPMALFORMED (silent drop)
- Brak write permissions do badRecordsPath
- FAILFAST mode zatrzymuje przed zapisem

**Rozwiązanie:**
```python
# Upewnij się że:
1. mode='PERMISSIVE'  # NOT 'DROPMALFORMED' or 'FAILFAST'
2. badRecordsPath ma write permissions
3. columnNameOfCorruptRecord jest ustawiony

# Przykład poprawnej konfiguracji:
df = spark.read \
  .format("csv") \
  .option("header", "true") \
  .option("mode", "PERMISSIVE") \
  .option("columnNameOfCorruptRecord", "_corrupt_record") \
  .option("badRecordsPath", "/tmp/bad_records") \
  .load(path)

# Sprawdź corrupt records w tabeli:
df.filter(col("_corrupt_record").isNotNull()).show()
```

### Problem 4: Slow Performance

**Objawy:**
- COPY INTO trwa godziny zamiast minut
- Spark UI pokazuje małe tasks (< 100MB)

**Przyczyna:**
- Małe pliki (< 128MB)
- Over-partitioning
- Zbyt mało executors

**Rozwiązanie 1: Coalesce małe pliki**
```python
# Pre-process: łącz małe pliki
df = spark.read.parquet("path/to/small/files")
df.coalesce(10).write.parquet("path/to/optimized/files")

# Następnie COPY INTO z optymalizowanych plików
```

**Rozwiązanie 2: Zwiększ parallelism**
```sql
-- Więcej shuffle partitions
SET spark.sql.shuffle.partitions = 200;

-- Auto-optimize w Delta Lake
ALTER TABLE table_name 
SET TBLPROPERTIES ('delta.autoOptimize.optimizeWrite' = 'true');
```

**Rozwiązanie 3: Użyj większych executors**
```python
# W cluster config:
spark.executor.memory = 16g
spark.executor.cores = 4
num_executors = 10
```

### Problem 5: Out of Memory (OOM)

**Objawy:**
```
OutOfMemoryError: Java heap space
Container killed by YARN for exceeding memory limits
```

**Przyczyna:**
- Bardzo duże pliki (> 2GB)
- Skewed data w MERGE operations
- Broadcast joins z dużymi tables

**Rozwiązanie:**
```python
# 1. Disable broadcast joins
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

# 2. Repartition przed zapisem
df.repartition(200).write.mode("append").saveAsTable("table")

# 3. Process w batches
for batch in file_batches:
    spark.read.parquet(batch).write.mode("append").saveAsTable("table")

# 4. Zwiększ executor memory
spark.executor.memory = 32g
spark.executor.memoryOverhead = 4g
```

### Problem 6: Lakeflow Connect Failures

**Objawy:**
- Connector job fails z timeout
- "Connection refused" errors
- Incremental sync stuck

**Rozwiązanie:**
```
1. Check credentials:
   - Unity Catalog → External Locations → Verify credentials
   - Test connection w Lakeflow UI

2. Network connectivity:
   - Whitelist Databricks IPs w firewall
   - Check VPN/Private Link configuration

3. Source system health:
   - Verify source database/API is accessible
   - Check rate limits (API throttling)

4. Retry configuration:
   - Increase retry attempts: 5 → 10
   - Increase timeout: 5min → 15min

5. Logs:
   - Databricks UI → Workflows → Lakeflow → View Logs
   - Check detailed error messages
```

### Debugging Checklist

Gdy coś nie działa, sprawdź po kolei:

✅ **1. Permissions**
```sql
-- Sprawdź permissions na katalog/schema
SHOW GRANTS ON SCHEMA bronze;
SHOW GRANTS ON TABLE bronze.customers;
```

✅ **2. File Path**
```python
# Upewnij się że ścieżka istnieje
dbutils.fs.ls("file:/path/to/data")
```

✅ **3. Schema**
```sql
-- Porównaj schema source vs target
DESCRIBE source_table;
DESCRIBE target_table;
```

✅ **4. Transaction Log**
```sql
-- Zobacz ostatnie operacje
DESCRIBE HISTORY target_table LIMIT 10;
```

✅ **5. Spark UI**
- Otwórz Spark UI (Clusters → Spark UI)
- Sprawdź "Stages" tab: czas wykonania, data skew
- Sprawdź "Executors" tab: memory usage, failed tasks

✅ **6. Logs**
```python
# Enable debug logging
spark.sparkContext.setLogLevel("DEBUG")

# Check driver logs
# Databricks: Clusters → Driver Logs
```

### Quick Debug Commands

```python
# 1. Count records w tabeli
spark.table("table_name").count()

# 2. Sample data
spark.table("table_name").show(5, truncate=False)

# 3. Check partitions
spark.table("table_name").select("partition_col").distinct().count()

# 4. Table size
spark.sql("DESCRIBE DETAIL table_name").select("sizeInBytes").show()

# 5. Recent operations
spark.sql("DESCRIBE HISTORY table_name").show(10, truncate=False)

# 6. Check files
dbutils.fs.ls("dbfs:/path/to/table")
```


### Co zostało osiągnięte:
- ✓ Idempotent batch loading z COPY INTO
- ✓ Obsługa różnych formatów (CSV, JSON, Parquet)
- ✓ Schema management (inference vs enforcement)
- ✓ Error handling (badRecordsPath, rescue columns)
- ✓ CTAS patterns dla transformacji
- ✓ Lakeflow Connect overview

## Sekcja 10: Czyszczenie Zasobów

**Uwaga:** Ta sekcja jest opcjonalna. Uruchom tylko jeśli chcesz usunąć wszystkie dane utworzone podczas notebooka.

W środowisku szkoleniowym zazwyczaj chcemy **zachować** dane dla kolejnych notebooków.


**Wyjaśnienie:**

COPY INTO z SELECT pozwala na:
- Transformację danych podczas ładowania (CAST, TO_DATE)
- Dodanie audit columns (_ingestion_timestamp, _source_file)
- Użycie _metadata kolumn (file_path, file_name, file_size)

Kolumny metadanych:
- `_metadata.file_path`: Pełna ścieżka do source file
- `_metadata.file_name`: Nazwa pliku
- `_metadata.file_size`: Rozmiar pliku w bajtach
- `_metadata.file_modification_time`: Data modyfikacji

Sprawdź załadowane dane:


In [ ]:
# Sprawdź załadowane dane z audit columns
print(f"📊 Dane w tabeli {TABLE_ORDERS}:")
display(spark.table(TABLE_ORDERS).limit(5))

# Sprawdź unikalne pliki źródłowe
print("\n📁 Źródła danych:")
spark.table(TABLE_ORDERS).select("_source_file").distinct().show(truncate=False)

# Statystyki
count = spark.table(TABLE_ORDERS).count()
print(f"\n✓ Łącznie załadowano {count} zamówień")


## Sekcja 2: File Formats

**Wprowadzenie teoretyczne:**

Delta Lake wspiera wiele formatów plików jako źródła danych. Wybór formatu wpływa na wydajność i łatwość użycia.

**Kluczowe różnice:**

| Format | Use Case | Pros | Cons |
|--------|----------|------|------|
| CSV | Legacy data, exports | Universal, human-readable | No schema, slow |
| JSON | APIs, nested data | Flexible schema, nested | Slower than Parquet |
| Parquet | Analytics, Delta Lake | Columnar, fast, compression | Not human-readable |
| Avro | Schema evolution, Kafka | Row-based, schema in file | Slower for analytics |

**Zalecenia:**
- **Production pipelines**: Używaj Parquet dla najlepszej wydajności
- **API integration**: JSON dla nested structures
- **Legacy systems**: CSV z explicit schema
- **Kafka streaming**: Avro z schema registry


### Przykład 2.1: COPY INTO z Parquet

**Cel:** Załadować dane produktów z formatu Parquet - najbardziej efektywny format dla analytics.

**Podejście:**
1. Utwórz tabelę dla produktów
2. Załaduj dane z Parquet używając COPY INTO
3. Porównaj wydajność z CSV/JSON


In [ ]:
# Przykład 2.1 - COPY INTO z Parquet

TABLE_PRODUCTS = f"{BRONZE_SCHEMA}.products_batch"

# Krok 1: Utwórz tabelę
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TABLE_PRODUCTS} (
  product_id STRING,
  product_name STRING,
  category STRING,
  unit_price DECIMAL(10,2),
  stock_quantity INT,
  supplier_id STRING,
  _ingestion_timestamp TIMESTAMP
) USING DELTA
""")

print(f"✓ Tabela {TABLE_PRODUCTS} gotowa")

# Krok 2: COPY INTO z Parquet (najszybszy format)
result = spark.sql(f"""
COPY INTO {TABLE_PRODUCTS}
FROM (
  SELECT 
    product_id,
    product_name,
    category,
    CAST(unit_price AS DECIMAL(10,2)) as unit_price,
    stock_quantity,
    supplier_id,
    current_timestamp() as _ingestion_timestamp
  FROM '{PRODUCTS_PARQUET}'
)
FILEFORMAT = PARQUET
""")

display(result)

# Krok 3: Weryfikacja
count = spark.table(TABLE_PRODUCTS).count()
print(f"\n✓ Załadowano {count} produktów")

# Pokaż przykładowe dane
display(spark.table(TABLE_PRODUCTS).limit(5))


**Wyjaśnienie:**

Parquet to columnar format:
- **Compression**: Automatyczna kompresja (3-10x mniejsze pliki)
- **Predicate pushdown**: Spark czyta tylko potrzebne kolumny
- **Schema embedded**: Metadata w pliku, nie trzeba definiować
- **Fastest queries**: Idealny dla analytics workloads

Porównanie wydajności (typowy scenariusz 100GB danych):
- CSV: ~5 min read time
- JSON: ~3 min read time  
- Parquet: ~30 sec read time

💡 **Best Practice**: Konwertuj CSV/JSON → Parquet w Bronze layer!


## Sekcja 3: Schema Management

**Wprowadzenie teoretyczne:**

Schema management to krytyczny aspekt data ingestion. Masz dwa główne podejścia:

**1. Schema Inference** (automatyczne wykrywanie):
- ✅ Pros: Szybkie, nie wymaga definicji
- ❌ Cons: Może być nieprecyzyjne, różne typy w różnych plikach
- 🎯 Use case: Exploration, ad-hoc analysis

**2. Schema Enforcement** (explicit definition):
- ✅ Pros: Precyzyjne typy, walidacja, dokumentacja
- ❌ Cons: Wymaga ręcznej definicji
- 🎯 Use case: Production pipelines, data quality

**Schema Evolution:**
- `mergeSchema=true`: Automatyczne dodawanie nowych kolumn
- `overwriteSchema=true`: Nadpisanie całego schema (ostrożnie!)

**Best Practices:**
- Production: Zawsze używaj explicit schema
- Development: Inference jest OK dla prototyping
- Add audit columns: _ingestion_timestamp, _source_file
- Document schema changes w version control


### Przykład 3.1: Schema Inference vs Enforcement

**Cel:** Porównać schema inference z explicit schema enforcement - zobaczyć różnice w typach danych.

**Podejście:**
1. Załaduj dane z schema inference
2. Załaduj te same dane z explicit schema
3. Porównaj wyniki i typy danych


## Sekcja 4: Error Handling & Data Quality

**Wprowadzenie teoretyczne:**

W real-world scenarios dane nie zawsze są idealne. Potrzebujemy strategii obsługi błędów:

**Strategie Error Handling:**

**1. Parse Mode Options:**
- `PERMISSIVE` (default): Parsuje co się da, błędne dane → _corrupt_record
- `DROPMALFORMED`: Usuwa błędne rekordy (silent failure - ostrożnie!)
- `FAILFAST`: Zatrzymuje całą operację na pierwszym błędzie

**2. badRecordsPath:**
- Zapisuje niepoprawne rekordy do osobnego folderu
- Umożliwia późniejszą analizę i naprawę
- Rekomendowane dla production

**3. Rescue Columns:**
- `_rescued_data`: Automatyczna kolumna dla danych, które nie pasują do schema
- Dostępne w JSON/CSV format
- Pozwala na analizę post-factum

**Best Practices:**
- Zawsze używaj `badRecordsPath` w production
- Monitoruj bad records folder (alerting)
- Mode PERMISSIVE + manual validation > DROPMALFORMED
- Loguj statystyki: success/failed records


### Przykład 4.1: Error Handling z badRecordsPath

**Cel:** Skonfigurować robust error handling przy ładowaniu danych - quarantine bad records.

**Podejście:**
1. Skonfiguruj badRecordsPath
2. Załaduj dane z PERMISSIVE mode
3. Sprawdź bad records folder
4. Przeanalizuj błędne dane


## Sekcja 5: CTAS (CREATE TABLE AS SELECT)

**Wprowadzenie teoretyczne:**

CTAS to powerful pattern do tworzenia tabel z zapytań. W przeciwieństwie do COPY INTO, CTAS:
- Tworzy nową tabelę w jednej operacji
- NIE jest idempotentne (każde uruchomienie tworzy/nadpisuje)
- Idealne do transformacji i agregacji

**CTAS vs COPY INTO vs INSERT:**

| Feature | CTAS | COPY INTO | INSERT |
|---------|------|-----------|--------|
| Idempotency | ❌ No | ✅ Yes | ❌ No |
| Use Case | One-time load | Incremental | Append data |
| Performance | Fast (parallel) | Fast | Slower |
| Schema | From SELECT | Pre-defined | Pre-defined |

**Kiedy używać CTAS:**
1. **Jednorazowe ładowanie**: Historyczny load z data lake
2. **Transformacje**: Tworzenie Silver/Gold tables z Bronze
3. **Agregacje**: Summary tables, reports
4. **Table cloning**: Backup, testing environments
5. **Format conversion**: CSV → Delta, Parquet → Delta

**CTAS Syntax:**
```sql
CREATE [OR REPLACE] TABLE table_name
[USING DELTA]
[PARTITIONED BY (col1, col2)]
[LOCATION 'path']
AS
SELECT ... FROM source_table
```


### Przykład 5.1: CTAS dla agregacji danych

**Cel:** Utworzyć summary table używając CTAS - agregacja customers per segment.

**Podejście:**
1. Użyj CTAS z GROUP BY do utworzenia aggregate table
2. Dodaj timestamp snapshot
3. Pokaż różnicę z COPY INTO


### Przykład 5.2: CTAS z transformacją (Bronze → Silver)

**Cel:** Użyć CTAS do transformacji danych z Bronze do Silver layer z data quality improvements.

**Podejście:**
1. Filtruj niepoprawne dane
2. Standaryzuj formaty (uppercase, trim)
3. Dodaj business logic columns


In [ ]:
# Przykład 5.2 - CTAS Bronze → Silver transformation

SILVER_CUSTOMERS = f"{SILVER_SCHEMA}.customers_clean"

print(f"=== Transformacja Bronze → Silver: {SILVER_CUSTOMERS} ===\n")

# CTAS z data quality improvements
spark.sql(f"""
CREATE OR REPLACE TABLE {SILVER_CUSTOMERS}
USING DELTA
COMMENT 'Cleaned customer data - Silver layer'
AS
SELECT 
  customer_id,
  TRIM(UPPER(first_name)) as first_name,
  TRIM(UPPER(last_name)) as last_name,
  CONCAT(first_name, ' ', last_name) as full_name,
  LOWER(TRIM(email)) as email,
  phone,
  address,
  UPPER(city) as city,
  UPPER(state) as state,
  zip_code,
  customer_segment,
  registration_date,
  DATEDIFF(CURRENT_DATE(), registration_date) as days_since_registration,
  CASE 
    WHEN customer_segment = 'Premium' THEN 'High Value'
    WHEN customer_segment = 'Standard' THEN 'Medium Value'
    ELSE 'Low Value'
  END as value_tier,
  current_timestamp() as processed_timestamp
FROM {TABLE_NAME}
WHERE 
  customer_id IS NOT NULL
  AND email IS NOT NULL
  AND email LIKE '%@%'
""")

print(f"✓ Silver table {SILVER_CUSTOMERS} utworzona\n")

# Porównaj Bronze vs Silver
bronze_count = spark.table(TABLE_NAME).count()
silver_count = spark.table(SILVER_CUSTOMERS).count()
filtered_out = bronze_count - silver_count

print(f"=== Statystyki transformacji ===")
print(f"Bronze records: {bronze_count}")
print(f"Silver records: {silver_count}")
print(f"Filtered out (bad quality): {filtered_out}")
print(f"Quality rate: {(silver_count/bronze_count*100):.2f}%\n")

# Pokaż dane Silver
print("=== Przykładowe dane Silver ===")
display(spark.table(SILVER_CUSTOMERS).limit(5))


## Sekcja 6: Lakeflow Connect (Managed Connectors)

**Wprowadzenie teoretyczne:**

Lakeflow Connect to managed integration platform w Databricks, która upraszcza łączenie z zewnętrznymi źródłami danych.

**Co to jest Lakeflow Connect:**
- **Managed connectors**: Bez konieczności pisania custom code
- **No-code/low-code**: Konfiguracja przez UI
- **Automatic schema inference**: Automatyczne wykrywanie schema
- **Built-in CDC**: Change Data Capture dla incremental sync
- **Unity Catalog integration**: Automatyczne tworzenie tabel

**Typy connectorów:**

**1. Fully-Managed Connectors** (SaaS/Enterprise):
- Salesforce (Objects, Reports, Custom)
- SQL Server, MySQL, PostgreSQL, Oracle
- Google Analytics 4, Google Sheets
- ServiceNow, Workday
- SAP, Snowflake
- SharePoint, OneDrive

**2. Standard Connectors** (Cloud Services):
- Kafka, Azure Event Hubs, AWS Kinesis
- S3, ADLS Gen2, Google Cloud Storage
- MongoDB, Cassandra
- REST APIs (generic)

**Kluczowe cechy:**
- ✅ Automatic retry & error handling
- ✅ Incremental sync (only new/changed data)
- ✅ Schema evolution support
- ✅ Built-in monitoring & alerting
- ✅ Security: credential management w Unity Catalog

**Kiedy używać:**
- Integracja z SaaS applications (Salesforce, ServiceNow)
- CDC z transactional databases (SQL Server, MySQL)
- Alternative do custom ETL code
- Szybkie PoC/MVP data pipelines


### Przykład 6.1: Lakeflow Connect - Setup Guide (UI-based)

**UWAGA:** Lakeflow Connect konfiguruje się przez Databricks UI, nie przez kod. Poniżej przedstawiamy konceptualny flow i przykładową konfigurację.

**Setup Process:**


## Sekcja 7: Best Practices - Batch Data Ingestion

**Wprowadzenie:**

Po poznaniu wszystkich technik, oto kluczowe best practices dla production-ready batch ingestion.

### 7.1 Wydajność i Optymalizacja

**File Size Optimization:**
```
Idealne rozmiary plików:
• Minimum: 128 MB per file
• Optimum: 256 MB - 1 GB per file  
• Maximum: < 1 GB per file
```

**Partycjonowanie:**
- Partycjonuj według najczęściej używanych filtrów (data, region)
- Unikaj over-partitioning (< 1GB per partition)
- Używaj Hive-style partitioning: `/year=2024/month=01/`

**Parallelizm:**
```sql
-- Zwiększ parallelism dla dużych loadów
spark.conf.set("spark.sql.shuffle.partitions", "200")

-- Coalesce małe pliki przed zapisem
df.coalesce(10).write.mode("append").saveAsTable("table")
```

### 7.2 Data Quality & Governance

**Obowiązkowe kolumny (Audit Trail):**
```python
.withColumn("_ingestion_timestamp", F.current_timestamp())
.withColumn("_source_file", F.input_file_name())
.withColumn("_ingestion_user", F.lit(spark.sparkContext.sparkUser()))
.withColumn("_job_id", F.lit(dbutils.notebook.entry_point.getDbutils().notebook().getContext().jobId().get()))
```

**Data Validation:**
- NOT NULL constraints na kluczowe kolumny
- CHECK constraints (Delta Lake 3.0+)
- Quarantine bad records (badRecordsPath)
- Monitoruj data quality metrics

**Unity Catalog:**
- Taguj tabele: PII, retention policy, data classification
- Dokumentuj schema: COMMENT na tabelach i kolumnach
- Kontroluj dostęp: GRANT SELECT/MODIFY per role

### 7.3 Idempotency Patterns

**Pattern 1: COPY INTO** (Recommended)
```sql
-- Automatyczna idempotency
COPY INTO table FROM 'path/*.parquet'
```

**Pattern 2: MERGE with watermark**
```sql
MERGE INTO target
USING source
ON target.id = source.id 
  AND source.date >= '2024-01-01' -- watermark
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
```

**Pattern 3: Overwrite partition**
```sql
INSERT OVERWRITE TABLE target 
PARTITION (date = '2024-01-15')
SELECT * FROM source WHERE date = '2024-01-15'
```

### 7.4 Error Handling & Monitoring

**Comprehensive Error Strategy:**
```python
try:
    # Main ingestion logic
    result = spark.sql("COPY INTO ...")
    
    # Log success metrics
    rows_inserted = result.select("num_affected_rows").collect()[0][0]
    logger.info(f"Success: {rows_inserted} rows")
    
except Exception as e:
    # Log error details
    logger.error(f"Ingestion failed: {str(e)}")
    
    # Send alert
    send_alert(team="data-eng", message=str(e))
    
    # Quarantine bad data
    # (automatically handled by badRecordsPath)
    
    # Decide: retry or fail
    if is_retryable(e):
        retry_with_backoff()
    else:
        raise
```

**Monitoring Checklist:**
- ✅ Rows ingested per run
- ✅ Ingestion duration (SLA: < 5 min)
- ✅ Bad records count (alert if > 1%)
- ✅ File counts (new files detected)
- ✅ Schema changes detected

### 7.5 Code Quality

**Template: Production Ingestion Function**
```python
def ingest_batch_data(
    source_path: str,
    target_table: str,
    file_format: str = "parquet",
    mode: str = "copy_into",
    validation_rules: dict = None
):
    """
    Production-ready batch ingestion function.
    
    Args:
        source_path: Cloud storage path (s3://, abfss://)
        target_table: Fully qualified table name
        file_format: csv, json, parquet, avro
        mode: copy_into (idempotent) or append
        validation_rules: Data quality checks
    """
    
    # Validation
    assert spark.catalog.tableExists(target_table), f"Table {target_table} doesn't exist"
    
    # Audit columns
    if mode == "copy_into":
        result = spark.sql(f"""
            COPY INTO {target_table}
            FROM (
                SELECT 
                    *,
                    current_timestamp() as _ingestion_timestamp,
                    _metadata.file_path as _source_file
                FROM '{source_path}'
            )
            FILEFORMAT = {file_format}
            COPY_OPTIONS ('mergeSchema' = 'false')
        """)
    
    # Log metrics
    return {
        "rows_inserted": result.select("num_affected_rows").first()[0],
        "timestamp": datetime.now(),
        "status": "success"
    }
```

### 7.6 Quick Reference Card

| Scenario | Recommended Approach |
|----------|---------------------|
| Incremental loads (new files daily) | COPY INTO |
| One-time historical load | CTAS or spark.read + write |
| SaaS integration (Salesforce, etc.) | Lakeflow Connect |
| High-frequency (< 1 hour) | Consider Streaming (Auto Loader) |
| Complex transformations | CTAS (Bronze → Silver) |
| Upserts (UPDATE + INSERT) | MERGE INTO |
| Full refresh | CREATE OR REPLACE TABLE |

### 7.7 Common Anti-Patterns (Unikaj!)

❌ **Anti-Pattern 1: INSERT bez idempotency**
```sql
-- BAD: Duplikaty przy ponownym uruchomieniu
INSERT INTO table SELECT * FROM source
```

✅ **Better: COPY INTO lub MERGE**

❌ **Anti-Pattern 2: Schema inference w production**
```python
# BAD: Typy mogą się zmienić między runami
df = spark.read.csv(path, inferSchema=True)
```

✅ **Better: Explicit schema**

❌ **Anti-Pattern 3: Brak error handling**
```python
# BAD: Silent failures
df.write.mode("append").saveAsTable("table")
```

✅ **Better: try/except + badRecordsPath + monitoring**

❌ **Anti-Pattern 4: Małe pliki (< 100MB)**
```python
# BAD: Tworzy tysiące małych plików
for file in files:
    spark.read.csv(file).write.mode("append")
```

✅ **Better: Batch processing + coalesce**


## Sekcja 9: Podsumowanie & Następne Kroki

### Co zostało osiągnięte w tym notebooku:

✅ **1. COPY INTO - Idempotent Batch Loading**
- Nauka idempotentnego ładowania danych bez duplikacji
- File tracking w Delta transaction log
- Transformacje podczas COPY INTO (SELECT)
- Audit columns: _ingestion_timestamp, _source_file

✅ **2. File Formats**
- CSV: universal ale wolny
- JSON: flexible dla nested data
- Parquet: najszybszy dla analytics (compression, columnar)
- Wybór formatu według use case

✅ **3. Schema Management**
- Schema Inference vs Explicit Schema
- Zalecenie: explicit schema dla production
- Schema evolution z mergeSchema
- Data type validation

✅ **4. Error Handling**
- badRecordsPath dla quarantine
- Parse modes: PERMISSIVE, DROPMALFORMED, FAILFAST
- Rescue columns (_corrupt_record)
- Monitoring bad records

✅ **5. CTAS Patterns**
- CREATE TABLE AS SELECT dla transformacji
- Bronze → Silver data quality improvements
- Aggregate tables (summary views)
- Różnice: CTAS vs COPY INTO vs INSERT

✅ **6. Lakeflow Connect**
- Managed connectors dla SaaS/databases
- No-code integration przez UI
- Automatic CDC dla incremental sync
- Use cases: Salesforce, SQL Server, etc.

✅ **7. Best Practices**
- Wydajność: file sizes, partitioning, parallelism
- Governance: Unity Catalog, tagging, audit trail
- Idempotency patterns
- Code quality templates

✅ **8. Troubleshooting**
- File already processed errors
- Schema mismatch resolution
- Bad records debugging
- Performance tuning
- Common pitfalls & solutions

### Kluczowe wnioski:

💡 **1. COPY INTO > INSERT**
```
Zawsze preferuj COPY INTO dla batch loads:
- Automatic idempotency
- File tracking
- No duplicates na retry
```

💡 **2. Explicit Schema > Inference**
```
Production pipelines wymagają:
- Zdefiniowany schema (CREATE TABLE)
- CAST transformations
- Data type validation
```

💡 **3. Always Error Handling**
```
Każdy production pipeline musi mieć:
- badRecordsPath
- PERMISSIVE mode
- Monitoring & alerting
- Retry logic
```

💡 **4. Audit Everything**
```
Dodawaj metadata columns:
- _ingestion_timestamp
- _source_file
- _ingestion_user
- _job_id
```

💡 **5. Optimize for Performance**
```
Best practices:
- Parquet > CSV/JSON
- Files: 256MB - 1GB
- Partition wisely
- Auto-optimize włączone
```

### Quick Reference - Decision Tree

```
Mam nowe dane do załadowania...

├─ Czy to jednorazowy load historycznych danych?
│  └─ TAK → Użyj CTAS
│     CREATE TABLE AS SELECT * FROM source
│
├─ Czy to incremental load (nowe pliki regularnie)?
│  └─ TAK → Użyj COPY INTO
│     COPY INTO table FROM 'path/*.parquet'
│
├─ Czy to SaaS/database integration?
│  └─ TAK → Użyj Lakeflow Connect
│     UI → Workflows → Create Lakeflow
│
├─ Czy to high-frequency (< 1h) micro-batches?
│  └─ TAK → Użyj Auto Loader (streaming)
│     spark.readStream.format("cloudFiles")
│
└─ Czy to upserts (UPDATE + INSERT)?
   └─ TAK → Użyj MERGE INTO
      MERGE INTO target USING source ON ...
```

### Comparison Matrix

| Feature | COPY INTO | CTAS | INSERT | Lakeflow |
|---------|-----------|------|--------|----------|
| **Idempotency** | ✅ Yes | ❌ No | ❌ No | ✅ Yes |
| **Use Case** | Incremental | One-time | Append | SaaS/CDC |
| **Code Required** | SQL | SQL | SQL | No-code UI |
| **Performance** | Fast | Fast | Medium | Fast |
| **File Tracking** | ✅ Auto | ❌ No | ❌ No | ✅ Auto |
| **CDC Support** | ❌ No | ❌ No | ❌ No | ✅ Yes |
| **Schema Evolution** | ✅ Optional | ✅ From SELECT | ❌ No | ✅ Auto |
| **Best For** | Daily/hourly loads | Transformations | Small appends | Managed integrations |

### Następne kroki w szkoleniu:

**📚 Kolejny Notebook:**
- **03_streaming_data_ingestion.ipynb**
  - Auto Loader (cloudFiles)
  - Structured Streaming
  - Micro-batch processing
  - Stream-to-Delta patterns

**🛠️ Warsztat Praktyczny:**
- **02_ingestion_pipeline_workshop.ipynb**
  - Hands-on: Build end-to-end ingestion pipeline
  - Real-world scenario z multiple sources
  - Error handling implementation
  - Monitoring & alerting setup

**📖 Materiały Dodatkowe:**
- Delta Lake Documentation: COPY INTO
- Databricks Blog: Lakeflow Connect Best Practices
- Unity Catalog: Governance Guide

### Zadanie Domowe (Optional):

**Zadanie:** Zbuduj complete batch ingestion pipeline dla własnych danych

**Requirements:**
1. ✅ Use COPY INTO dla idempotency
2. ✅ Explicit schema definition (no inference)
3. ✅ Error handling z badRecordsPath
4. ✅ Audit columns (_ingestion_timestamp, _source_file)
5. ✅ CTAS transformation Bronze → Silver
6. ✅ Data quality checks (NOT NULL, valid ranges)
7. ✅ Unity Catalog tagging (PII, retention)
8. ✅ Monitoring metrics (success rate, row counts)

**Bonus:**
- Schedule jako Databricks Job (daily/hourly)
- Integrate z Lakeflow Connect (jeśli masz SaaS source)
- Add alerting on failures

### Feedback & Questions

Masz pytania? Znalazłeś błąd? 

📧 Kontakt: instruktor@example.com
📝 Issues: github.com/company/databricks-training

---

**Gratulacje!** 🎉 
Ukończyłeś notebook o Batch Data Ingestion. 
Jesteś gotowy do pracy z production-grade data pipelines w Delta Lake!


### Opcja 1: Zachowaj dane (zalecane dla szkolenia)

Zostaw tabele dla kolejnych notebooków:
- `02_medallion_architecture.ipynb` będzie używał tych danych
- `04_bronze_silver_gold_pipeline.ipynb` transformuje te tabele
- Warsztaty praktyczne również korzystają z Bronze tables


In [ ]:
# Opcja 1: Sprawdź utworzone zasoby (bez usuwania)

print("=== Utworzone tabele w tym notebooku ===\n")

created_tables = [
    f"{BRONZE_SCHEMA}.customers_batch",
    f"{BRONZE_SCHEMA}.orders_batch",
    f"{BRONZE_SCHEMA}.products_batch",
    f"{BRONZE_SCHEMA}.customers_with_validation",
    f"{SILVER_SCHEMA}.customer_segment_summary",
    f"{SILVER_SCHEMA}.customers_clean"
]

total_size_bytes = 0

for table in created_tables:
    full_table = f"{CATALOG}.{table}"
    try:
        if spark.catalog.tableExists(full_table):
            count = spark.table(full_table).count()
            
            # Pobierz rozmiar
            detail = spark.sql(f"DESCRIBE DETAIL {full_table}").collect()[0]
            size_bytes = detail['sizeInBytes']
            size_mb = size_bytes / (1024 * 1024)
            total_size_bytes += size_bytes
            
            print(f"✅ {table}")
            print(f"   Rekordy: {count:,}")
            print(f"   Rozmiar: {size_mb:.2f} MB")
            print()
        else:
            print(f"⚠️  {table} - nie istnieje")
            print()
    except Exception as e:
        print(f"⚠️  {table} - błąd: {str(e)}")
        print()

total_size_mb = total_size_bytes / (1024 * 1024)
print(f"{'='*60}")
print(f"Łączny rozmiar: {total_size_mb:.2f} MB")
print(f"{'='*60}")

print("\n💡 Dane są zachowane dla kolejnych notebooków")
print("💡 Aby usunąć, uruchom komórkę poniżej (Opcja 2)")


### Opcja 2: Usuń wszystkie zasoby (tylko jeśli naprawdę chcesz)

**UWAGA:** To usunie wszystkie tabele i dane utworzone w tym notebooku!

Uruchom komórkę poniżej tylko jeśli:
- Skończyłeś szkolenie i chcesz posprzątać
- Chcesz zacząć od nowa (fresh start)
- Testujesz notebook i potrzebujesz clean slate


In [ ]:
# Opcja 2: Usuń wszystkie zasoby
# UWAGA: To jest DESTRUKCYJNE! Odkomentuj tylko jeśli jesteś pewien!

CLEANUP_ENABLED = False  # Zmień na True aby włączyć cleanup

if CLEANUP_ENABLED:
    print("🗑️  Usuwanie zasobów...\n")
    
    # Lista tabel do usunięcia
    tables_to_drop = [
        f"{CATALOG}.{BRONZE_SCHEMA}.customers_batch",
        f"{CATALOG}.{BRONZE_SCHEMA}.orders_batch",
        f"{CATALOG}.{BRONZE_SCHEMA}.products_batch",
        f"{CATALOG}.{BRONZE_SCHEMA}.customers_with_validation",
        f"{CATALOG}.{SILVER_SCHEMA}.customer_segment_summary",
        f"{CATALOG}.{SILVER_SCHEMA}.customers_clean"
    ]
    
    dropped_count = 0
    
    for table in tables_to_drop:
        try:
            spark.sql(f"DROP TABLE IF EXISTS {table}")
            print(f"✅ Usunięto: {table}")
            dropped_count += 1
        except Exception as e:
            print(f"⚠️  Błąd przy usuwaniu {table}: {str(e)}")
    
    # Usuń folder bad records
    try:
        BAD_RECORDS_PATH = f"/tmp/{raw_user}/bad_records"
        dbutils.fs.rm(BAD_RECORDS_PATH, True)
        print(f"\n✅ Usunięto bad records: {BAD_RECORDS_PATH}")
    except Exception as e:
        print(f"\n⚠️  Błąd przy usuwaniu bad records: {str(e)}")
    
    print(f"\n{'='*60}")
    print(f"✅ Cleanup zakończony!")
    print(f"   Usuniętych tabel: {dropped_count}")
    print(f"{'='*60}")
    
else:
    print("⚠️  Cleanup WYŁĄCZONY")
    print("   Aby włączyć, zmień CLEANUP_ENABLED = True")
    print("   i uruchom komórkę ponownie")
    print("\n💡 Zalecane: Zostaw dane dla kolejnych notebooków!")


---

## 🎉 Gratulacje!

Ukończyłeś notebook **02_batch_data_ingestion.ipynb**!

**Nauczyłeś się:**
- ✅ COPY INTO dla idempotentnego ładowania danych
- ✅ Obsługi różnych formatów plików (CSV, JSON, Parquet)
- ✅ Schema management (inference vs enforcement)
- ✅ Error handling z badRecordsPath
- ✅ CTAS patterns dla transformacji
- ✅ Lakeflow Connect dla managed integrations
- ✅ Best practices dla production pipelines

**Następne kroki:**
- 📖 **Kolejny notebook:** `02_medallion_architecture.ipynb` - Bronze/Silver/Gold patterns
- 🛠️ **Warsztat:** `02_ingestion_pipeline_workshop.ipynb` - Hands-on exercise
- 📚 **Dokumentacja:** Delta Lake COPY INTO reference

**Pytania?** Skontaktuj się z instruktorem lub sprawdź dokumentację Databricks.

---

**Notebook:** `02_batch_data_ingestion.ipynb`  
**Czas trwania:** ~45 minut  
**Poziom:** Intermediate  
**Ostatnia aktualizacja:** Listopad 2024
